# Bibliotecas 

In [1]:
import pandas as pd
import numpy as np
import re
from pycaret.classification import *

# Importação das Informações 

In [2]:
#importação das partidas 
partidas_2017 = pd.read_csv('data/2017_partidas.csv')
partidas_2016 = pd.read_csv('data/2016_partidas.csv')
partidas_2015 = pd.read_csv('data/2015_partidas.csv')
partidas_2014 = pd.read_csv('data/2014_partidas.csv')

In [3]:
#importação dos scouts
scouts_2017 = pd.read_csv('data/2017_scouts.csv')
scouts_2016 = pd.read_csv('data/2016_scouts.csv')
scouts_2015 = pd.read_csv('data/2015_scouts.csv')
scouts_2014 = pd.read_csv('data/2014_scouts.csv')

In [4]:
#importação dos clubes 
clubes_2017 = pd.read_csv('data/2017_clubes.csv')
clubes_2016 = pd.read_csv('data/2016_clubes.csv')
clubes_2015 = pd.read_csv('data/2015_clubes.csv')
clubes_2014 = pd.read_csv('data/2014_clubes.csv')

In [5]:
#importação dos atletas
atletas_2017 = pd.read_csv('data/2017_atletas.csv')
atletas_2016 = pd.read_csv('data/2016_atletas.csv')
atletas_2015 = pd.read_csv('data/2015_atletas.csv')
atletas_2014 = pd.read_csv('data/2014_atletas.csv')

# Junção das Tabelas

In [6]:
partidas = partidas_2017

# Transformação de Dados

In [7]:
#como no dataset de partidas não fala o resultado(vitória, empate ou derrota)
partidas['resultado'] = 0
partidas.loc[partidas['placar_oficial_mandante'] > partidas['placar_oficial_visitante'],'resultado'] = 1
partidas.loc[partidas['placar_oficial_mandante'] < partidas['placar_oficial_visitante'],'resultado'] = -1

In [8]:
#transformação dos aproveitamentos 
partidas['vitorias_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('v')
partidas['empates_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('e')
partidas['derrotas_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('d')
partidas['vitorias_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('v')
partidas['empates_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('e')
partidas['derrotas_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('d')

In [9]:
#transformação para int64 da coluna valida
partidas['valida'] = partidas['valida'].astype('int64')

In [10]:
#removendo vazamento de dados e dados já transformados
partidas.drop(columns=['aproveitamento_visitante','aproveitamento_mandante','placar_oficial_visitante','placar_oficial_mandante'], inplace=True)

In [11]:
#remoção de colunas inuteis para o modelo
partidas.drop(columns=['partida_data', 'local','clube_casa_id','clube_visitante_id'],inplace=True)

In [21]:
partidas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   rodada_id                    200 non-null    int64
 1   clube_casa_posicao           200 non-null    int64
 2   clube_visitante_posicao      200 non-null    int64
 3   valida                       200 non-null    int64
 4   resultado                    200 non-null    int64
 5   vitorias_recentes_mandante   200 non-null    int64
 6   empates_recentes_mandante    200 non-null    int64
 7   derrotas_recentes_mandante   200 non-null    int64
 8   vitorias_recentes_visitante  200 non-null    int64
 9   empates_recentes_visitante   200 non-null    int64
 10  derrotas_recentes_visitante  200 non-null    int64
dtypes: int64(11)
memory usage: 17.3 KB


In [24]:
#transformação para int32 do dataframe
partidas = partidas.astype('int32')

# Modelagem

In [25]:
classification = setup(data = partidas, target='resultado',session_id=42)

,Description,Value
0,session_id,42
1,Target,resultado
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(200, 11)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [26]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6473,0.8163,0.6271,0.6798,0.6478,0.4571,0.4677,0.0310
ridge,Ridge Classifier,0.6324,0.0000,0.6076,0.6487,0.6192,0.4266,0.4415,0.0030
lda,Linear Discriminant Analysis,0.6324,0.8354,0.6087,0.6682,0.6301,0.4318,0.4431,0.0050
lightgbm,Light Gradient Boosting Machine,0.6110,0.7835,0.5929,0.6255,0.6042,0.4025,0.4112,0.0130
gbc,Gradient Boosting Classifier,0.5962,0.8020,0.5674,0.5943,0.5839,0.3730,0.3801,0.0440
dt,Decision Tree Classifier,0.5890,0.6996,0.5729,0.6334,0.5945,0.3801,0.3903,0.0030
rf,Random Forest Classifier,0.5824,0.7831,0.5604,0.6036,0.5654,0.3579,0.3719,0.0530
et,Extra Trees Classifier,0.5819,0.7832,0.5667,0.5841,0.5596,0.3567,0.3710,0.0410
nb,Naive Bayes,0.5681,0.8270,0.5421,0.4933,0.4933,0.3298,0.3706,0.0030
ada,Ada Boost Classifier,0.5615,0.8161,0.5295,0.5626,0.5487,0.3269,0.3362,0.0220


In [15]:
print(best_model)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [27]:
model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7857,0.8232,0.7500,0.8214,0.7721,0.6613,0.6849
1,0.5714,0.7518,0.5556,0.6000,0.5815,0.3538,0.3566
2,0.7143,0.9179,0.6944,0.7619,0.7243,0.5692,0.5827
3,0.6429,0.7839,0.6389,0.6571,0.6450,0.4615,0.4651
4,0.8571,0.8768,0.8333,0.8673,0.8548,0.7778,0.7842
5,0.6429,0.8714,0.6389,0.7143,0.6476,0.4531,0.4680
6,0.7143,0.8299,0.6444,0.6888,0.6956,0.5447,0.5495
7,0.3571,0.6841,0.3444,0.3571,0.3549,0.0156,0.0157
8,0.5714,0.8590,0.6151,0.6571,0.5901,0.3588,0.3791


In [28]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [29]:
final_model = finalize_model(model)